Qual é o produto mais consumido?

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col, sum

spark = SparkSession.builder \
    .appName("RankingProdutos") \
    .getOrCreate()

In [14]:
df_vendas_atual = spark.read.parquet("../../datalake/silver/Liquidos_Vendas_Atual")

In [15]:
df_vendas_atual = df_vendas_atual.withColumn(
    "quantidade_de_produto_(mil_m3)",
    regexp_replace("quantidade_de_produto_(mil_m3)", ",", ".").cast("double")
)

In [16]:
ranking_produtos = df_vendas_atual.groupBy("nome_do_produto") \
    .agg(sum("quantidade_de_produto_(mil_m3)").alias("quantidade_total_(mil_m3)")) \
    .orderBy(col("quantidade_total_(mil_m3)").desc())

ranking_produtos.show(truncate=False)

+----------------+-------------------------+
|nome_do_produto |quantidade_total_(mil_m3)|
+----------------+-------------------------+
|Diesel B        |505312.9506719748        |
|Gasolina C      |344083.4166619903        |
|Etanol Hidratado|152310.2439409996        |
|Óleo Comb.      |18916.003599999996       |
+----------------+-------------------------+



In [17]:
ranking_produtos.write.mode("overwrite").parquet("../../datalake/gold/Liquidos_Ranking_Produtos")